## import libraries

In [1]:
import numpy as np
from sklearn import linear_model, preprocessing, metrics, kernel_ridge, model_selection
import matplotlib.pyplot as plt

## import data and standardize

In [2]:
scaler_x = preprocessing.StandardScaler()
scaler_y = preprocessing.StandardScaler()

Train = np.loadtxt('wine_training.csv', delimiter = ',')
train_x = scaler_x.fit_transform(Train[:, 0:11])
train_y = Train[:,11].reshape(-1,1)
train_y = scaler_y.fit_transform(train_y)

Test = np.loadtxt('wine_test.csv', delimiter = ',')
test_x = scaler_x.transform(Test[:, 0:11])
test_y = Test[:,11].reshape(-1,1)
test_y = scaler_y.transform(test_y)

new_batch = scaler_x.transform(np.loadtxt('wine_new_batch.csv', delimiter = ','))


## Visualize data

In [3]:
features = ['fixed acidity', 'volatile acidity', 'citric acid','residual sugar','chlorides',
           'free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']

# generate some plots to visualize the data set 

fig, ax = plt.subplots(4,3, figsize=(24,24))

for j in range(4):
  for i in range(3):
    if i + j*3 < 11:
        ax[j][i].scatter( train_x[:, i+ j*3], train_y )
        ax[j][i].set_xlabel(str(features[i + j*3]), fontsize=20)
    else:
        ax[j][i].axis('off')

## Linear Regression

In [5]:
reg = linear_model.LinearRegression().fit(train_x, train_y)

Lin_train_MSE = metrics.mean_squared_error(train_y, reg.predict(train_x))

Lin_test_MSE = metrics.mean_squared_error(test_y, reg.predict(test_x))

In [6]:
print(Lin_train_MSE)
print(Lin_test_MSE)

0.6278484956554882
0.747169690518721


## Gaussian Kernel Regression

In [7]:
gaus_alpha = np.linspace(-3,-2.5,10)
gaus_gamma = np.linspace(-5.02,-4.9,10)

In [8]:
parameters = {'kernel':['rbf'], 'gamma':2**gaus_gamma, 'alpha':2**gaus_alpha}
ker_reg = kernel_ridge.KernelRidge()
clf_gauss = model_selection.GridSearchCV(ker_reg, parameters, cv = 10, return_train_score = True, 
                                         scoring = "neg_mean_squared_error", n_jobs = -1)
clf_gauss.fit(train_x, train_y)

GridSearchCV(cv=10, estimator=KernelRidge(), n_jobs=-1,
             param_grid={'alpha': array([0.125     , 0.1299074 , 0.13500747, 0.14030776, 0.14581613,
       0.15154076, 0.15749013, 0.16367307, 0.17009875, 0.1767767 ]),
                         'gamma': array([0.03081977, 0.03110593, 0.03139474, 0.03168623, 0.03198043,
       0.03227737, 0.03257706, 0.03287953, 0.03318481, 0.03349292]),
                         'kernel': ['rbf']},
             return_train_score=True, scoring='neg_mean_squared_error')

In [9]:
print('alpha : ' + str(np.log2(clf_gauss.best_estimator_.alpha)))
print('alpha : ' + str(clf_gauss.best_estimator_.alpha))

print('gamma : ' + str(np.log2(clf_gauss.best_estimator_.gamma)))
print('gamma : ' + str(clf_gauss.best_estimator_.gamma))
print('sigma : ' + str(1/np.sqrt(2*clf_gauss.best_estimator_.gamma)))

alpha : -2.7222222222222223
alpha : 0.15154075835169306
gamma : -4.993333333333333
gamma : 0.03139473982506418
sigma : 3.9907687061080934


## Laplacian Kernel Regression

In [10]:
lap_alpha = np.linspace(-2.5,-2,10)
lap_gamma = np.linspace(-2.1,-2,10)

In [11]:
parameters = {'kernel':['laplacian'], 'gamma':2**lap_gamma, 'alpha':2**lap_alpha}
ker_reg = kernel_ridge.KernelRidge()
clf_lap = model_selection.GridSearchCV(ker_reg, parameters, cv = 10, return_train_score = True, 
                                       scoring = "neg_mean_squared_error", n_jobs = -1)
clf_lap.fit(train_x, train_y)

GridSearchCV(cv=10, estimator=KernelRidge(), n_jobs=-1,
             param_grid={'alpha': array([0.1767767 , 0.18371681, 0.19092939, 0.19842513, 0.20621515,
       0.214311  , 0.22272468, 0.23146868, 0.24055596, 0.25      ]),
                         'gamma': array([0.23325825, 0.23506165, 0.236879  , 0.2387104 , 0.24055596,
       0.24241579, 0.24428999, 0.24617869, 0.24808199, 0.25      ]),
                         'kernel': ['laplacian']},
             return_train_score=True, scoring='neg_mean_squared_error')

In [12]:
print('alpha : ' + str(np.log2(clf_lap.best_estimator_.alpha)))
print('alpha : ' + str(clf_lap.best_estimator_.alpha))
print('gamma : ' + str(np.log2(clf_lap.best_estimator_.gamma)))
print('gamma : ' + str(clf_lap.best_estimator_.gamma))
print('sigma : ' + str(1/np.sqrt(2*clf_lap.best_estimator_.gamma)))

alpha : -2.1666666666666665
alpha : 0.22272467953508485
gamma : -2.022222222222222
gamma : 0.24617868823360783
sigma : 1.4251473695757046


In [13]:
pred_train_gauss = clf_gauss.predict(train_x)
pred_test_gauss = clf_gauss.predict(test_x)

pred_train_lap = clf_lap.predict(train_x)
pred_test_lap = clf_lap.predict(test_x)

In [14]:
MSE_train_gauss = metrics.mean_squared_error(train_y, pred_train_gauss)
MSE_test_gauss = metrics.mean_squared_error(test_y, pred_test_gauss)

MSE_train_lap = metrics.mean_squared_error(train_y, pred_train_lap)
MSE_test_lap = metrics.mean_squared_error(test_y, pred_test_lap)



In [15]:
print('Linear regression train MSE = ' + str(Lin_train_MSE))
print('Linear regression test MSE = ' + str(Lin_test_MSE))
print('MSE_train_gauss = ' + str(MSE_train_gauss))
print('MSE_test_gauss = ' + str(MSE_test_gauss))
print('MSE_train_lap = ' + str(MSE_train_lap))
print('MSE_train_lap = ' + str(MSE_test_lap))

Linear regression train MSE = 0.6278484956554882
Linear regression test MSE = 0.747169690518721
MSE_train_gauss = 0.45826976506129075
MSE_test_gauss = 0.6834997030985077
MSE_train_lap = 0.05111511239858154
MSE_train_lap = 0.6067475712133398


In [16]:
new_batch_lin = reg.predict(new_batch)
new_batch_gauss = clf_gauss.predict(new_batch)
new_batch_lap = clf_lap.predict(new_batch)


In [17]:
new_batch_lin_pred = scaler_y.inverse_transform(new_batch_lin)
new_batch_gauss_pred = scaler_y.inverse_transform(new_batch_gauss)
new_batch_lap_pred = scaler_y.inverse_transform(new_batch_lap)

In [18]:
print('new batch linear regression predicton: ' + str(new_batch_lin_pred))
print('new batch gaussian predicton: ' + str(new_batch_gauss_pred))
print('new batch laplacian predicton: ' + str(new_batch_lap_pred))

new batch linear regression predicton: [[6.00469789]
 [5.28767761]
 [5.56363072]
 [6.067022  ]
 [5.94248207]]
new batch gaussian predicton: [[5.97874758]
 [5.43930772]
 [5.33399219]
 [6.11520117]
 [6.04010049]]
new batch laplacian predicton: [[6.03438545]
 [5.48735329]
 [5.62171015]
 [5.95472088]
 [5.99109488]]


## Gauss Contour plot

In [19]:
K_gamma = 10
K_lmbd = 10

lmbd = np.linspace(-3, -2.5, K_lmbd)
gamma = np.linspace(-5.5, -4.5, K_gamma)

scores = np.zeros((K_gamma, K_lmbd))
scores_std = np.zeros((K_gamma, K_lmbd))

KRR_CV = kernel_ridge.KernelRidge(kernel='rbf')

for i in range(K_gamma):

  KRR_CV.gamma = 2**gamma[i]

  for j in range(K_lmbd): 

    KRR_CV.alpha = (2**lmbd[j])
    this_score = model_selection.cross_val_score(KRR_CV, train_x, train_y, cv=10)

    scores[i,j] = np.abs(np.mean(this_score))
    scores_std[i,j] = np.abs(np.std(this_score))


print(scores)

[[0.3931865  0.39303614 0.39287777 0.39271175 0.39253843 0.39235816
  0.39217129 0.39197818 0.39177916 0.39157457]
 [0.39386725 0.39375678 0.39363689 0.39350791 0.39337022 0.39322414
  0.39307003 0.39290823 0.39273909 0.39256294]
 [0.39430647 0.39424008 0.39416302 0.39407562 0.39397822 0.39387114
  0.39375474 0.39362934 0.39349527 0.39335286]
 [0.39451323 0.39449337 0.39446179 0.39441881 0.39436474 0.39429991
  0.39422462 0.3941392  0.39404395 0.3939392 ]
 [0.39450732 0.39453483 0.39454978 0.39455249 0.39454323 0.3945223
  0.39449    0.39444662 0.39439244 0.39432777]
 [0.39431487 0.39438942 0.39445077 0.39449919 0.39453495 0.39455832
  0.39456957 0.39456897 0.3945568  0.39453331]
 [0.3939629  0.39408369 0.39419073 0.39428425 0.39436453 0.3944318
  0.39448635 0.3945284  0.39455822 0.39457606]
 [0.39347365 0.39364023 0.39379245 0.39393058 0.39405487 0.39416558
  0.39426296 0.39434725 0.39441871 0.39447758]
 [0.39285989 0.39307291 0.39327082 0.39345391 0.39362247 0.39377676
  0.39391704 0

sv, lv = np.meshgrid(gamma, lmbd)

fig1, ax1 = plt.subplots(1,2, figsize=(20,10))

cm0 = ax1[0].contourf(2**((-sv-1)/2), 2**lv, scores)
ax1[0].set_xlabel('$\sigma$')
ax1[0].set_ylabel('$\lambda$')
ax1[0].set_title('MSE')
fig1.colorbar(cm0, ax=ax1[0])


cm1 = ax1[1].contourf(2**((-sv-1)/2), 2**lv, scores_std)
ax1[1].set_xlabel('$\sigma$')
ax1[1].set_ylabel('$\lambda$')
ax1[1].set_title('MSE std')
fig1.colorbar(cm1, ax=ax1[1])


fig1.tight_layout()

## Laplacian Countour plot

In [21]:
K_gamma = 10
K_lmbd = 10

lmbd = np.linspace(-2.4, -1.9, K_lmbd)
gamma = np.linspace(-2.1, -1.9, K_gamma)

scores = np.zeros((K_gamma, K_lmbd))
scores_std = np.zeros((K_gamma, K_lmbd))

KRR_CV = kernel_ridge.KernelRidge(kernel='laplacian')

for i in range(K_gamma):

  KRR_CV.gamma = 2**gamma[i]

  for j in range(K_lmbd): 

    KRR_CV.alpha = (2**lmbd[j])
    this_score = model_selection.cross_val_score(KRR_CV, train_x, train_y, cv=10)

    scores[i,j] = np.abs(np.mean(this_score))
    scores_std[i,j] = np.abs(np.std(this_score))


print(scores)

[[0.44881235 0.44889679 0.44896566 0.44901822 0.4490537  0.44907135
  0.44907042 0.44905018 0.44900989 0.44894884]
 [0.44892354 0.44900105 0.44906311 0.44910896 0.44913784 0.44914899
  0.44914166 0.4491151  0.4490686  0.44900142]
 [0.44902268 0.44909318 0.44914833 0.44918737 0.44920954 0.44921407
  0.44920022 0.44916723 0.44911437 0.44904092]
 [0.44910911 0.44917251 0.44922065 0.44925278 0.44926813 0.44926594
  0.44924544 0.4492059  0.44914656 0.4490667 ]
 [0.44918211 0.44923833 0.44927939 0.44930451 0.44931294 0.44930391
  0.44927666 0.44923043 0.44916449 0.44907809]
 [0.44924099 0.44928994 0.44932382 0.44934184 0.44934325 0.44932728
  0.44929316 0.44924014 0.44916747 0.4490744 ]
 [0.44928499 0.44932661 0.44935322 0.44936406 0.44935835 0.44933533
  0.44929424 0.4492343  0.44915478 0.44905492]
 [0.44931337 0.44934758 0.44936685 0.44937041 0.44935749 0.44932733
  0.44927915 0.44921219 0.4491257  0.44901892]
 [0.44932535 0.44935208 0.44936393 0.44936013 0.44933992 0.44930251
  0.44924715

sv, lv = np.meshgrid(gamma, lmbd)

#fig = plt.figure(figsize = (10, 7))
#ax = plt.axes(projection ="3d")

#ax.scatter(sv, lv, np.log(scores))
#ax.set_xlabel('$\sigma$', fontsize=20)
#ax.set_ylabel('$\lambda$', fontsize=20)

fig2, ax2 = plt.subplots(1,2, figsize=(20,10))

cm0 = ax2[0].contourf(2**((-sv-1)/2), 2**lv, scores)
ax2[0].set_xlabel('$\sigma$')
ax2[0].set_ylabel('$\lambda$')
ax2[0].set_title('MSE')
fig2.colorbar(cm0, ax=ax2[0])


cm1 = ax2[1].contourf(2**((-sv-1)/2), 2**lv, scores_std)
ax2[1].set_xlabel('$\sigma$')
ax2[1].set_ylabel('$\lambda$')
ax2[1].set_title('MSE std')
fig2.colorbar(cm1, ax=ax2[1])


fig2.tight_layout()

In [23]:
scores_std

array([[0.05422037, 0.05416397, 0.05410818, 0.05405308, 0.05399872,
        0.05394517, 0.0538925 , 0.05384076, 0.05379001, 0.0537403 ],
       [0.05415538, 0.05409957, 0.05404433, 0.05398973, 0.05393583,
        0.05388269, 0.05383038, 0.05377896, 0.05372847, 0.05367897],
       [0.05409473, 0.0540395 , 0.05398479, 0.05393066, 0.0538772 ,
        0.05382445, 0.05377248, 0.05372135, 0.05367111, 0.05362181],
       [0.05403847, 0.05398379, 0.05392958, 0.05387592, 0.05382287,
        0.05377049, 0.05371884, 0.05366799, 0.05361797, 0.05356886],
       [0.05398663, 0.05393248, 0.05387875, 0.05382553, 0.05377288,
        0.05372085, 0.0536695 , 0.0536189 , 0.0535691 , 0.05352014],
       [0.05393925, 0.05388559, 0.05383233, 0.05377952, 0.05372724,
        0.05367553, 0.05362447, 0.05357411, 0.05352449, 0.05347568],
       [0.05389632, 0.05384314, 0.05379031, 0.0537379 , 0.05368597,
        0.05363457, 0.05358377, 0.05353362, 0.05348418, 0.05343549],
       [0.05385785, 0.05380512, 0.0537527

In [28]:
#fig.savefig('features.png')
#fig1.savefig('gauss_hyp.png')
#fig2.savefig('lap_hyp.png')